In [1]:
# Import dependcies
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Create global seed
yogi = 8

In [2]:
# Read in data 
file_path = '../Data/Pitchers/nola.csv'
nola_df = pd.read_csv(file_path)

nola_df.dropna(inplace = True)

nola_df.head(10)

,Unnamed: 0,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,...,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp
0,4007,KC,78.7,-1.75,5.01,"Nola, Aaron",3.0,R,L,R,...,79.1,2514.0,7.1,53.40,36,8,Knuckle Curve,61.0,0.009,0.200
1,4008,FF,91.9,-1.76,4.86,"Nola, Aaron",11.0,R,L,R,...,93.1,2125.0,7.3,53.24,36,7,4-Seam Fastball,235.0,0.000,0.062
2,4009,CH,84.6,-1.91,4.79,"Nola, Aaron",13.0,R,L,R,...,85.1,1541.0,6.8,53.72,36,6,Changeup,263.0,0.000,0.039
3,4010,FF,93.5,-1.74,4.88,"Nola, Aaron",1.0,R,L,R,...,94.6,2195.0,7.3,53.23,36,5,4-Seam Fastball,233.0,0.000,0.000
4,4011,KC,79.2,-1.75,4.94,"Nola, Aaron",5.0,R,L,R,...,79.6,2590.0,7.2,53.32,36,4,Knuckle Curve,56.0,0.000,0.000
5,4012,KC,79.0,-1.81,4.93,"Nola, Aaron",13.0,R,L,R,...,79.0,2526.0,6.9,53.64,36,3,Knuckle Curve,57.0,0.000,-0.057
6,4013,CH,85.2,-1.99,4.75,"Nola, Aaron",14.0,R,L,R,...,85.7,1569.0,6.9,53.61,36,2,Changeup,259.0,0.000,0.023
7,4014,CH,84.7,-1.97,4.77,"Nola, Aaron",7.0,R,L,R,...,85.8,1448.0,7.3,53.22,36,1,Changeup,255.0,0.000,-0.041
8,4015,KC,77.1,-1.75,5.04,"Nola, Aaron",11.0,R,L,R,...,77.2,2481.0,6.9,53.62,35,7,Knuckle Curve,58.0,-0.012,-0.314
9,4016,FF,92.5,-1.69,4.91,"Nola, Aaron",11.0,R,L,R,...,93.3,2178.0,7.1,53.35,35,6,4-Seam Fastball,230.0,0.000,0.065


In [3]:
# Drop Unnamed columns
nola_df.drop(['Unnamed: 0', 'pitch_name'], axis = 1, inplace = True)

In [4]:
# Generate categorical variable list
pitch_cat = nola_df.dtypes[nola_df.dtypes == 'object'].index.tolist()
pitch_cat.remove('type')
pitch_cat.remove('player_name')
pitch_cat

['pitch_type', 'game_type', 'stand', 'p_throws']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(nola_df[pitch_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(pitch_cat)
encode_df.head()

,pitch_type_CH,pitch_type_FF,pitch_type_FT,pitch_type_KC,pitch_type_SI,game_type_R,stand_L,stand_R,p_throws_R
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
nola_df = nola_df.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(pitch_cat, 1)

nola_df.head()

,release_speed,release_pos_x,release_pos_z,player_name,zone,type,balls,strikes,pfx_x,pfx_z,...,delta_run_exp,pitch_type_CH,pitch_type_FF,pitch_type_FT,pitch_type_KC,pitch_type_SI,game_type_R,stand_L,stand_R,p_throws_R
0,78.7,-1.75,5.01,"Nola, Aaron",3.0,B,3,2,1.32,-0.80,...,0.200,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,91.9,-1.76,4.86,"Nola, Aaron",11.0,B,2,2,-1.16,1.09,...,0.062,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,84.6,-1.91,4.79,"Nola, Aaron",13.0,B,1,2,-1.43,-0.02,...,0.039,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,93.5,-1.74,4.88,"Nola, Aaron",1.0,S,1,2,-1.36,1.18,...,0.000,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,79.2,-1.75,4.94,"Nola, Aaron",5.0,S,1,2,1.13,-1.07,...,0.000,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [7]:
# Drop superfluous columns 
nola_df.drop(columns = ['stand_L'], axis = 1, inplace = True)
nola_df.head()


,release_speed,release_pos_x,release_pos_z,player_name,zone,type,balls,strikes,pfx_x,pfx_z,...,delta_home_win_exp,delta_run_exp,pitch_type_CH,pitch_type_FF,pitch_type_FT,pitch_type_KC,pitch_type_SI,game_type_R,stand_R,p_throws_R
0,78.7,-1.75,5.01,"Nola, Aaron",3.0,B,3,2,1.32,-0.80,...,0.009,0.200,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,91.9,-1.76,4.86,"Nola, Aaron",11.0,B,2,2,-1.16,1.09,...,0.000,0.062,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,84.6,-1.91,4.79,"Nola, Aaron",13.0,B,1,2,-1.43,-0.02,...,0.000,0.039,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,93.5,-1.74,4.88,"Nola, Aaron",1.0,S,1,2,-1.36,1.18,...,0.000,0.000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,79.2,-1.75,4.94,"Nola, Aaron",5.0,S,1,2,1.13,-1.07,...,0.000,0.000,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


### Compiling, Training, and Testing Data

In [8]:
# Define features set
X = nola_df.copy()
X.drop(columns = ['player_name', 'type', 'balls', 'strikes', 'delta_run_exp', 'delta_home_win_exp'], axis = 1, inplace = True)
X.head()

,release_speed,release_pos_x,release_pos_z,zone,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,...,pitch_number,spin_axis,pitch_type_CH,pitch_type_FF,pitch_type_FT,pitch_type_KC,pitch_type_SI,game_type_R,stand_R,p_throws_R
0,78.7,-1.75,5.01,3.0,1.32,-0.80,0.64,3.24,4,2.782614,...,8,61.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,91.9,-1.76,4.86,11.0,-1.16,1.09,-0.26,3.75,4,6.434379,...,7,235.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,84.6,-1.91,4.79,13.0,-1.43,-0.02,-0.83,0.84,4,5.419228,...,6,263.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,93.5,-1.74,4.88,1.0,-1.36,1.18,-0.65,3.10,4,5.936697,...,5,233.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,79.2,-1.75,4.94,5.0,1.13,-1.07,0.06,2.16,4,1.879353,...,4,56.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [9]:
# Define target vector
y = nola_df['type'].values
y[:5]

array(['B', 'B', 'B', 'S', 'S'], dtype=object)

In [10]:
# Split the data into training and testing sets - stratify by pitcher
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = yogi)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit Standard Scaler 
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Find Best Learning Rate

In [12]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
best_rate = 0
best_acc = 0

for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(
        n_estimators = 20,
        learning_rate = learning_rate,
        max_features = 7,
        max_depth = 4,
        random_state = yogi)
    
    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print(f'Learning Rate: {learning_rate}')

    # Score the model
    print('Accuracy Score (training): {0:.3f}'.format(
        classifier.score(
            X_train_scaled,
            y_train
        )
    ))
    print('Accuracy Score (validation): {0:.3f}'.format(
        classifier.score(
            X_test_scaled,
            y_test
        )
    ))
    print()

    if classifier.score(X_test_scaled, y_test) > best_acc:
        best_acc = classifier.score(X_test_scaled, y_test)
        best_rate = learning_rate

Learning Rate: 0.05
Accuracy Score (training): 0.737
Accuracy Score (validation): 0.704

Learning Rate: 0.1
Accuracy Score (training): 0.740
Accuracy Score (validation): 0.710

Learning Rate: 0.25
Accuracy Score (training): 0.785
Accuracy Score (validation): 0.703

Learning Rate: 0.5
Accuracy Score (training): 0.845
Accuracy Score (validation): 0.700

Learning Rate: 0.75
Accuracy Score (training): 0.882
Accuracy Score (validation): 0.683

Learning Rate: 1
Accuracy Score (training): 0.897
Accuracy Score (validation): 0.661



### Create model with best learning rate

In [13]:
# Choose a learning rate and create classifiers
classifier = GradientBoostingClassifier(
    n_estimators = 20,
    learning_rate = best_rate,
    max_features = 7,
    max_depth = 3,
    random_state = yogi
)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make predictions
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).head(20)

,Prediction,Actual
0,S,S
1,S,S
2,B,B
3,S,S
4,B,B
5,S,S
6,S,X
7,B,B
8,B,B
9,S,S


### Evaluate Model

In [14]:
# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.7066189624329159


In [15]:
# Generate classification report
print('Classification Report')
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           B       0.77      0.81      0.79       377
           S       0.67      0.87      0.76       554
           X       0.00      0.00      0.00       187

    accuracy                           0.71      1118
   macro avg       0.48      0.56      0.52      1118
weighted avg       0.59      0.71      0.64      1118



C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
